In [5]:
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
import os
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import TensorBoard

# Define the path where your keypoint data is stored
DATA_PATH = os.path.join('MP_Data')

# Get the list of actions (labels)
actions = np.array(os.listdir(DATA_PATH))
print(f"Actions: {actions}")

# Create a label map
label_map = {label: num for num, label in enumerate(actions)}
print(f"Label Map: {label_map}")

# Initialize sequences and labels
sequences, labels = [], []

# Sequence length (number of frames per video)
# Adjust as needed based on your data analysis
sequence_length = 10

# Collect sequence lengths for analysis
sequence_lengths = []

for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    sequences_dirs = os.listdir(action_path)
    for sequence in sequences_dirs:
        window = []
        sequence_path = os.path.join(action_path, sequence)
        frame_files = os.listdir(sequence_path)
        frame_files = sorted(frame_files, key=lambda x: int(os.path.splitext(x)[0]))
        for frame_file in frame_files:
            res = np.load(os.path.join(sequence_path, frame_file))
            window.append(res)
        sequence_lengths.append(len(window))
        
        # Pad or truncate sequences to the desired length
        if len(window) < sequence_length:
            # Pad with zeros
            padding = [np.zeros_like(window[0])]*(sequence_length - len(window))
            window.extend(padding)
            sequences.append(window)
            labels.append(label_map[action])
        elif len(window) > sequence_length:
            # Truncate the sequence
            window = window[:sequence_length]
            sequences.append(window)
            labels.append(label_map[action])
        else:
            sequences.append(window)
            labels.append(label_map[action])

# Analyze sequence lengths
from collections import Counter
print("Sequence length distribution:", Counter(sequence_lengths))

# Convert to NumPy arrays
X = np.array(sequences)
y = to_categorical(labels).astype(int)

# Split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.05, random_state=42)

# Print shapes
print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

# Rest of your code remains the same


Actions: ['about' 'after' 'again' 'all' 'animal' 'ask' 'baby' 'back' 'bad'
 'because' 'bed' 'before' 'big' 'book' 'boy' 'bring' 'brother' 'buy'
 'call' 'can' 'car' 'chair' 'child' 'city' 'clothes' 'come' 'computer'
 'day' 'dog' 'drink' 'eat' 'family' 'father' 'find' 'first' 'friend' 'get'
 'give' 'go' 'good' 'happy' 'have' 'help' 'her' 'home' 'hot' 'how' 'know'
 'late' 'leave' 'like' 'make' 'man' 'many' 'me' 'mother' 'name' 'need'
 'new' 'no' 'now' 'on' 'one' 'out' 'people' 'play' 'please' 'read' 'right'
 'run' 'say' 'school' 'see' 'she' 'sit' 'sleep' 'small' 'sorry' 'stop'
 'study' 'take' 'thank you' 'think' 'time' 'today' 'tomorrow' 'want'
 'watch' 'we' 'what' 'when' 'where' 'who' 'why' 'work' 'yes' 'you']
Label Map: {'about': 0, 'after': 1, 'again': 2, 'all': 3, 'animal': 4, 'ask': 5, 'baby': 6, 'back': 7, 'bad': 8, 'because': 9, 'bed': 10, 'before': 11, 'big': 12, 'book': 13, 'boy': 14, 'bring': 15, 'brother': 16, 'buy': 17, 'call': 18, 'can': 19, 'car': 20, 'chair': 21, 'child': 2

In [7]:
# 7. Build and Train LSTM Neural Network

# Define the TensorBoard callback
log_dir = os.path.join('Logs')
if not os.path.exists(log_dir):
    os.makedirs(log_dir)
tb_callback = TensorBoard(log_dir=log_dir)

# Get the number of classes (actions)
num_classes = len(actions)

# Build the model
model = Sequential()
model.add(LSTM(64, return_sequences=True, activation='relu', input_shape=(sequence_length, X.shape[2])))
model.add(LSTM(128, return_sequences=True, activation='relu'))
model.add(LSTM(64, return_sequences=False, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(num_classes, activation='softmax'))

# Compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['categorical_accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=2000, callbacks=[tb_callback])

# 8. Make Predictions

# Make predictions on the test set
yhat = model.predict(X_test)

# Convert predictions and true labels from one-hot encoding to label indices
yhat_classes = np.argmax(yhat, axis=1)
ytrue = np.argmax(y_test, axis=1)

# Map indices back to action labels
predicted_actions = [actions[idx] for idx in yhat_classes]
true_actions = [actions[idx] for idx in ytrue]

# Compare predictions to true labels
for i in range(len(y_test)):
    print(f"Sequence {i+1}: Predicted - {predicted_actions[i]}, True - {true_actions[i]}")

# 9. Save Weights

# Save the model
model_save_path = 'action.h5'
model.save(model_save_path)
print(f"Model saved to {model_save_path}")

Epoch 1/2000
21/21 [==============================] - 5s 80ms/step - loss: 4.5801 - categorical_accuracy: 0.0075
Epoch 2/2000
21/21 [==============================] - 2s 77ms/step - loss: 4.5727 - categorical_accuracy: 0.0150
Epoch 3/2000
21/21 [==============================] - 1s 68ms/step - loss: 4.5741 - categorical_accuracy: 0.0135
Epoch 4/2000
21/21 [==============================] - 1s 61ms/step - loss: 4.5680 - categorical_accuracy: 0.0120
Epoch 5/2000
21/21 [==============================] - 1s 68ms/step - loss: 4.5628 - categorical_accuracy: 0.0165
Epoch 6/2000
21/21 [==============================] - 2s 73ms/step - loss: 4.5612 - categorical_accuracy: 0.0165
Epoch 7/2000
21/21 [==============================] - 2s 72ms/step - loss: 4.5527 - categorical_accuracy: 0.0150
Epoch 8/2000
21/21 [==============================] - 1s 71ms/step - loss: 4.5446 - categorical_accuracy: 0.0210
Epoch 9/2000
21/21 [==============================] - 2s 76ms/step - loss: 4.5410 - categorical_

In [4]:
import os
import numpy as np

DATA_PATH = os.path.join('MP_Data')
actions = np.array(os.listdir(DATA_PATH))

empty_files = []

for action in actions:
    action_path = os.path.join(DATA_PATH, action)
    sequences_dirs = os.listdir(action_path)
    for sequence in sequences_dirs:
        sequence_path = os.path.join(action_path, sequence)
        frame_files = os.listdir(sequence_path)
        for frame_file in frame_files:
            file_path = os.path.join(sequence_path, frame_file)
            try:
                data = np.load(file_path)
                if data.size == 0:
                    empty_files.append(file_path)
            except Exception as e:
                print(f"Error loading {file_path}: {e}")
                empty_files.append(file_path)

print(f"Found {len(empty_files)} empty or corrupted .npy files.")
for file in empty_files:
    print(file)


Found 0 empty or corrupted .npy files.
